In [1]:
%run TF-IDF_rawImplementation.ipynb

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
%%time
processed_doc_path = './LabShare/data/chui_ma/spacy_corpus'
tf_idf, word2index, index2document, inv_doc_freq = tf_idf_PrerocessedDoc(processed_doc_path, 100)

....................................................................................................CPU times: user 1.84 s, sys: 252 ms, total: 2.1 s
Wall time: 1.28 s


In [3]:
#defining the similarity function
from scipy import spatial
import numpy as np
def similarity(u,v):
    return 1 - spatial.distance.cosine(u, v)

In [4]:
# preprocssing the query vector
q = 'banken'
query = []
q_v = np.zeros(len(word2index))
for w in q.split():
    try:
        q_v[word2index[w]]+=1
        query.append(word2index[w])
    except:
        print('keywords not found')
sum_ = np.sum(q_v)
def normalize_query(i,sum_):
    return 0.5+(0.5*i)/sum_
norm_q = np.vectorize(normalize_query)
q_v = norm_q(q_v,sum_)

# normalizing the query
q_v = np.multiply(q_v,inv_doc_freq)

In [5]:
def find_top_k(k,doc_sim,index2document):
    for doc,sim in sorted(doc_sim.items(),key = lambda x:x[1], reverse=True):
        print (index2document[doc])
        k-=1
        if k==0:
            break

In [8]:
# latent semantic indexing (LSI)
from numpy import linalg

'''
    query:indices of words appear in matrix
    k:dimention of latent space
'''

def latent_semantic_indexing(query, tf_idf, k, topk, index2document):
    doc_similarity = {}
#     print(tf_idf)
    u,s,v = linalg.svd(tf_idf)
    u = u[:,:k]
    s_ = np.zeros((k,k))
    for i in range(k):
        s_[i,i] = s[i]
    v = v[:k,:]
    q_v = np.zeros(len(word2index))
    for q in query:
        q_v[q]+=1
    q_v = q_v.reshape(1,-1)
    q_v = np.matmul(q_v,u)
    s_ = linalg.inv(s_)
    q_v = np.matmul(q_v,s_)
    
    for i in range(v.shape[1]):
        sim = similarity(q_v,v[:,i])
        doc_similarity[i] = sim 
    find_top_k(topk, doc_similarity,index2document)

In [7]:
%%time
latent_semantic_indexing(query, tf_idf, 3, 5, index2document)

WCM-AnnualReport-2011_corpus.txt
RhoenKlinikum-AnnualReport-2013_corpus.txt
HambornerReit-QuarterlyReport-2016-Q1_corpus.txt
Bertrandt-AnnualReport-2015_corpus.txt
Bertrandt-QuarterlyReport-2012-Q2_corpus.txt
CPU times: user 16min 32s, sys: 32.9 s, total: 17min 5s
Wall time: 4min 27s
